In [4]:
!provengo --batch-mode create SheepWolfCabbage

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

20:16:01.230 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
20:16:01.301 INFO [SETUP] Project folder: c:\temp\ProvengoDemo\SheepWolfCabbage


In [ ]:
%%writefile SheepWolfCabbage/spec/js/hello-world.js

// Behavior for the sheep: The farmer can move the sheep between the left and right sides of the river.
bthread("Sheep", function () {
    while (true) {
        // Request to move the sheep from the left to the right side
        request(bp.Event("RowTheBoat", { with: "Sheep", from: "L", to: "R" }));
        // Request to move the sheep from the right to the left side
        request(bp.Event("RowTheBoat", { with: "Sheep", from: "R", to: "L" }));
    }
});

// Behavior for the cabbage: The farmer can move the cabbage between the left and right sides of the river.
bthread("Cabbage", function () {
    while (true) {
        // Request to move the cabbage from the left to the right side
        request(bp.Event("RowTheBoat", { with: "Cabbage", from: "L", to: "R" }));
        // Request to move the cabbage from the right to the left side
        request(bp.Event("RowTheBoat", { with: "Cabbage", from: "R", to: "L" }));
    }
});

// Behavior for the wolf: The farmer can move the wolf between the left and right sides of the river.
bthread("Wolf", function () {
    while (true) {
        // Request to move the wolf from the left to the right side
        request(bp.Event("RowTheBoat", { with: "Wolf", from: "L", to: "R" }));
        // Request to move the wolf from the right to the left side
        request(bp.Event("RowTheBoat", { with: "Wolf", from: "R", to: "L" }));
    }
});

// Behavior for the farmer: The farmer can move alone between the left and right sides of the river.
bthread("Farmer", function () {
    while (true) {
        // Request to move alone from the left to the right side
        // Wait for any event moving from left to right and block events moving in the opposite direction
        sync({ 
            request: bp.Event("RowTheBoat", { with: "No one", from: "L", to: "R" }), 
            waitFor: Any({ from: "L", to: "R" }), 
            block: Any({ from: "R", to: "L" }) 
        });

        // Request to move alone from the right to the left side
        // Wait for any event moving from right to left and block events moving in the opposite direction
        sync({ 
            request: bp.Event("RowTheBoat", { with: "No one", from: "R", to: "L" }), 
            waitFor: Any({ from: "R", to: "L" }), 
            block: Any({ from: "L", to: "R" }) 
        });
    }
});

// Safety constraint: Prevent the wolf and sheep from being left alone on the same side.
bthread("WolfSheepSafety", function () {
    const WolfSheepEvents = Any({ with: "Wolf" }).or(Any({ with: "Sheep" })); // Events involving the wolf or sheep

    while (true) {
        // Ensure the wolf and sheep are not left alone by blocking invalid moves
        sync({ waitFor: WolfSheepEvents, block: WolfSheepEvents.negate() });
        // Wait for the farmer to move the wolf or sheep to the other side
        waitFor(WolfSheepEvents);
    }
});

// Safety constraint: Prevent the sheep and cabbage from being left alone on the same side.
bthread("SheepCabbageSafety", function () {
    const SheepCabbageEvents = Any({ with: "Sheep" }).or(Any({ with: "Cabbage" })); // Events involving the sheep or cabbage

    while (true) {
        // Ensure the sheep and cabbage are not left alone by blocking invalid moves
        sync({ waitFor: SheepCabbageEvents, block: SheepCabbageEvents.negate() });
        // Wait for the farmer to move the sheep or cabbage to the other side
        waitFor(SheepCabbageEvents);
    }
});

// Goal specification: Track the state of all entities and ensure the goal conditions are met.
bthread("Goal Specification", function () {
    state = { Sheep: "L", Wolf: "L", Cabbage: "L" }; // Initial state: all entities are on the left side

    while (true) {
        // Wait for any event and update the state based on the event
        let e = waitFor(EventSets.all);
        state[e.data.with] = e.data.to;

        // Log the current state for debugging purposes
        bp.log.info("Current state: " + JSON.stringify(state));

        // Assert that the wolf, sheep, and cabbage are not all on the right side simultaneously
        bp.ASSERT(
            state["Wolf"] != "R" || state["Sheep"] != "R" || state["Cabbage"] != "R",
            "The wolf, sheep and cabbage are not all on the right side of the river"
        );
    }
});

Overwriting SheepWolfCabbage/spec/js/hello-world.js


In [109]:
!provengo run SheepWolfCabbage

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

21:59:41.060 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
21:59:41.062 INFO [SETUP] Project path: c:\temp\ProvengoDemo\SheepWolfCabbage
21:59:41.070 INFO [RUN] Preparing to run
21:59:41.399 INFO [RUN>random] B-program started
21:59:41.433 INFO [RUN>random] Selected: [RowTheBoat {with:"Sheep", from:"L", to:"R"}]
21:59:41.449 INFO [RUN>random] Current state: {"Sheep":"R","Wolf":"L","Cabbage":"L"}
21:59:41.455 INFO [RUN>random] Selected: [RowTheBoat {with:"No one", from:"R", to:"L"}]
21:59:41.463 INFO [RUN>random] Current state: {"Sheep":"R","Wolf":"L","Cabbage":"L","No one":"L"}
21:59:41.468 INFO [RUN>random] Se

In [76]:
!provengo verify SheepWolfCabbage

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

21:38:03.359 INFO [VERSION] 1b75ef8/20250331-0214/master
21:38:03.369 ERR  [Main] Terminating because of previous errors. See logs.
21:38:03.375 ERR  [Main] 40 BadRequest


usage: provengo [-h] [--verbose] [--batch-mode] [--no-color]
                [--config CONFIG] [-p PROFILE] [-c [CUSTOM]] [-v]
                {run,analyze,gen-scripts,report,sample,ensemble,gen-book,create}
                ...
provengo: error: invalid choice:  'verify'  (choose  from 'run', 'analyze',
'gen-scripts', 'report', 'sample', 'ensemble', 'gen-book', 'create')
